# Part 3 - Summarization

For Part 3, use the same 300-word long sections as in Part 1, but instead of answering the questions about these sections, ask the ChatPI to summarize them for you, using Hugging Face summarization pipeline.

Part 3.2 -- use two different HF summarization models: use the default summarization pipeline, then use other models of choice and discuss the differences in the result.

https://huggingface.co/docs/transformers/main_classes/pipelines

https://huggingface.co/docs/transformers/v4.35.0/en/main_classes/pipelines#transformers.SummarizationPipeline


In [ ]:
!pip3 install -r ../requirements.txt

%load_ext autoreload
%autoreload 2


In [10]:
from collections import defaultdict

from src import utils
from src.summarization import run, run_models
import evaluate

import spacy
from spacy.cli import download
import csv
import os

---
---

## Experiments & Results

Use the same 300-word long sections as in Part 1, but instead of answering the questions about these sections, ask the ChatPI to summarize them for you, using Hugging Face summarization pipeline.


In [11]:
models = [
    "sshleifer/distilbart-cnn-12-6",
    "slauw87/bart_summarisation",
    "pszemraj/pegasus-x-large-book-summary",
]

score_metrics = [
    "rouge", 
    "bleu",
    "bertscore",
    "spacy_sim",
]


rouge_metric_data = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
bleu_metric_data = ["bleu", "precisions", "brevity_penalty", "length_ratio", "translation_length", "reference_length"]
bert_metric_data = ["precision", "recall", "f1", "hashcode"]



In [12]:
ctxs = ["protagonist", "antagonist", "crime", "evidence", "resolution"]

csvfile = "res_sum.csv"

header = [
    "ctx_name",
    "ctx_fname",
    "ctx_text",
    "model_idx",
    "model_name",
    "answer_pred",
]

rows = []

# Will need to adjust column headers so we know which header is for which metric
for ctx_name in ctxs:
    for i, (fname, ctx) in enumerate(utils.read_context(ctx_name)):
        fname = os.path.basename(fname)
        print("#" * 80)
        print("#" * 80)
        print(ctx)
        
        scores_by_model = defaultdict(list)
        model_answers = defaultdict(list)
        
        # Loop through the metrics we are using
        for metric in score_metrics:
            if metric == "bertscore":
                answers, scores = run_models(
                    ctx, 
                    models=models, 
                    expected_answer=ctx, 
                    metric=metric, 
                    lang='en'
                )
            else:
                answers, scores = run_models(
                    ctx, 
                    models=models, 
                    expected_answer=ctx, 
                    metric=metric
                )
            
            for k, (m, a, s) in enumerate(zip(models, answers, scores)):
                scores_by_model[m].append(s)
                model_answers[m].append(a)
            
        for k, (m, s) in enumerate(scores_by_model.items()):
            row = [
                ctx_name,
                fname,
                ctx,
                k,
                m,
                model_answers[m][0],
            ]
            for i, val in enumerate(s):
                for metric, score in val.items():
                    if metric not in header:
                        if metric in rouge_metric_data:
                            metric = "Rouge: " + metric
                            if metric not in header: 
                                header.append(metric)
                        elif metric in bleu_metric_data:
                            metric = "Bleu: " + metric
                            if metric not in header: 
                                header.append(metric)
                        elif metric in bert_metric_data:
                            metric = "Bert: " + metric
                            if metric not in header: 
                                header.append(metric)
                        else:
                            header.append(metric)
                    row.append(score)
            rows.append(row)


with open(csvfile, "w") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)

Found: ['protagonist.0.md', 'protagonist.qa.md']
################################################################################
################################################################################
Sherlock Holmes took his bottle from the corner of the mantel-piece and his hypodermic syringe from its neat morocco case. With his long, white, nervous fingers he adjusted the delicate needle, and rolled back his left shirt-cuff. For some little time his eyes rested thoughtfully upon the sinewy forearm and wrist all dotted and scarred with innumerable puncture-marks. Finally he thrust the sharp point home, pressed down the tiny piston, and sank back into the velvet-lined arm-chair with a long sigh of satisfaction. Three times a day for many months I had witnessed this performance, but custom had not reconciled my mind to it. On the contrary, from day to day I had become more irritable at the sight, and my conscience swelled nightly within me at the thought that I had lacked the

Your max_length is set to 512, but your input_length is only 382. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=191)


> Sherlock Holmes took his bottle from the corner of the mantel-piece and his hypodermic syringe from its neat morocco case. With his long, white, nervous fingers he adjusted the delicate needle, and rolled back his left shirt-cuff. For some little time his eyes rested thoughtfully upon the sinewy forearm and wrist all dotted and scarred with innumerable puncture-marks. Finally he thrust the sharp point home, pressed down the tiny piston, and sank back into the velvet-lined arm-chair with a long sigh of satisfaction. Three times a day for many months I had witnessed this performance, but custom had not reconciled my mind to it. On the contrary, from day to day I had become more irritable at the sight, and my conscience swelled nightly within me at the thought that I had lacked the courage to protest. Again and again I had registered a vow that I should deliver my soul upon the subject, but there was that in the cool, nonchalant air of my companion which made him the last man with whom 

Your max_length is set to 512, but your input_length is only 382. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=191)


> Sherlock Holmes took his bottle from the corner of the mantel-piece and his hypodermic syringe from its neat morocco case. With his long, white, nervous fingers he adjusted the delicate needle, and rolled back his left shirt-cuff. For some little time his eyes rested thoughtfully upon the sinewy forearm and wrist all dotted and scarred with innumerable puncture-marks. Finally he thrust the sharp point home, pressed down the tiny piston, and sank back into the velvet-lined arm-chair with a long sigh of satisfaction. Three times a day for many months I had witnessed this performance, but custom had not reconciled my mind to it. On the contrary, from day to day I had become more irritable at the sight, and my conscience swelled nightly within me at the thought that I had lacked the courage to protest. Again and again I had registered a vow that I should deliver my soul upon the subject, but there was that in the cool, nonchalant air of my companion which made him the last man with whom 

...

> A map is drawn for them by an Englishman named Jonathan Small. You remember that we saw the name upon the chart in Captain Morstan's possession. He had signed it in behalf of himself and his associates,-the sign of the four, as he somewhat dramatically called it. Aided by this chart, the officers-or one of them-gets the treasure and brings it to England, leaving, we will suppose, some condition under which he received it unfulfilled. Now, then, why did not Jonathan Small get the treasure himself? The answer is obvious. The chart is dated at a time when Morstan was brought into close association with convicts. Jonathan Small did not get the treasure because he and his associates were themselves convicts and could not get away." "But that is mere speculation," said I. "It is more than that. It is the only hypothesis which covers the facts. Let us see how it fits in with the sequel. Major Sholto remains at peace for some years, happy in the possession of his treasure. Then he receives a

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--------------------------------------------------------------------------------
> A map is drawn for them by an Englishman named Jonathan Small. You remember that we saw the name upon the chart in Captain Morstan's possession. He had signed it in behalf of himself and his associates,-the sign of the four, as he somewhat dramatically called it. Aided by this chart, the officers-or one of them-gets the treasure and brings it to England, leaving, we will suppose, some condition under which he received it unfulfilled. Now, then, why did not Jonathan Small get the treasure himself? The answer is obvious. The chart is dated at a time when Morstan was brought into close association with convicts. Jonathan Small did not get the treasure because he and his associates were themselves convicts and could not get away." "But that is mere speculation," said I. "It is more than that. It is the only hypothesis which covers the facts. Let us see how it fits in with the sequel. Major Sholto remains at 

Your max_length is set to 512, but your input_length is only 373. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=186)


> A map is drawn for them by an Englishman named Jonathan Small. You remember that we saw the name upon the chart in Captain Morstan's possession. He had signed it in behalf of himself and his associates,-the sign of the four, as he somewhat dramatically called it. Aided by this chart, the officers-or one of them-gets the treasure and brings it to England, leaving, we will suppose, some condition under which he received it unfulfilled. Now, then, why did not Jonathan Small get the treasure himself? The answer is obvious. The chart is dated at a time when Morstan was brought into close association with convicts. Jonathan Small did not get the treasure because he and his associates were themselves convicts and could not get away." "But that is mere speculation," said I. "It is more than that. It is the only hypothesis which covers the facts. Let us see how it fits in with the sequel. Major Sholto remains at peace for some years, happy in the possession of his treasure. Then he receives a